In [383]:
import gmpy2
import numpy as np

class SilverPohligHellman:
    def __init__(self, a, b, p):
        self.a = a
        self.b = b
        self.p = p

    def factor_num(self, n):
        result = []
        if n < 0:
            result.append(-1)
            n = -n
        for i in range(2, np.uint64(np.floor((np.sqrt(n))) + 1)):
            while (n % i == 0):
                result.append(i)
                n //= i
        if (n != 1):
            result.append(n)
        return np.array(result)
    
    def calculate_r(self):
        self.q_list = list(q_pairs.keys())
        self.r = []
        for i in range(len(self.q_list)):
            r_temp = {}
            for j in range(self.q_list[i]):
                r_temp[j] = gmpy2.powmod(self.a, gmpy2.mpz(j * ((self.p - 1) / self.q_list[i])), self.p)
            self.r.append(r_temp)
    
    def calculate_x_s(self):
        self.x_s = []
        self.alpha_list = list(self.q_pairs.values())

        for i in range(len(self.q_list)):
            r_indexes = list(self.r[i].keys())
            r_values = list(self.r[i].values())
            temp = []
            for j in range(self.alpha_list[i]):
                if j == 0:
                    temp.append(r_indexes[len(r_values) - 1 - r_values[::-1].index(gmpy2.powmod(self.b, int((self.p - 1) / self.q_list[i]), self.p))])
                elif j == 1:
                    temp.append(r_indexes[len(r_values) - 1 - r_values[::-1].index(gmpy2.powmod(self.b * gmpy2.powmod(self.a, -temp[0], self.p), int((self.p - 1) / self.q_list[i] ** 2), self.p))])
                else:
                    temp.append(r_indexes[len(r_values) - 1 - r_values[::-1].index(gmpy2.powmod(self.b * gmpy2.powmod(self.a, int(-sum([temp[k] * self.q_list[i] ** (k) for k in range(j)])), self.p), int((self.p - 1) / self.q_list[i] ** (j + 1)), self.p))])

            self.x_s.append(temp)
    
    def make_equations(self):
        self.equations = []
        for i in range(len(self.x_s)):
            summ = 0
            for j in range(len(self.x_s[i])):
                summ += self.x_s[i][j] * (self.q_list[i] ** j)
            mod = self.q_list[i] ** self.alpha_list[i]
            self.equations.append([summ % mod, mod])
    
    def solve_system(self):
        self.M = 1
        self.m = []
        self.N = []
        for i in range(len(self.equations)):
            self.M *= self.equations[i][1]
            
        self.x = 0    
        for i in range(len(self.equations)):
            self.m.append(int(self.M / self.equations[i][1]))
            self.N.append(gmpy2.gcdext(int(self.m[i]), int(self.equations[i][1]))[1])
            self.x += (self.equations[i][0] * self.m[i] * self.N[i]) % self.M
        self.x %= self.M
    
    def do_all_stuff(self, verbose = True):
        self.q_s = self.factor_num(self.p - 1)
        self.q_set = np.array(list(set(self.q_s)))
        self.q_pairs = {}
        
        for i in range(len(q_set)):
            self.q_pairs[self.q_set[i]] = np.count_nonzero(self.q_s == self.q_set[i])

        self.calculate_r()
        self.calculate_x_s()
        self.make_equations()
        self.solve_system()
        
        if verbose:
            print('{0}^x = {1}mod{2}'.format(self.a, self.b, self.p))
            s = ''
            for i in range(len(self.q_list)):
                if i != len(self.q_list) - 1:
                    s += '{0}^{1} + '.format(self.q_list[i], self.alpha_list[i])
                else:
                    s += '{0}^{1}'.format(self.q_list[i], self.alpha_list[i])
                    
            print('p - 1 = {0}'.format(s))
            for i in range(len(self.x_s)):
                print('\n{0}\nq = {1}'.format('#' * 10, self.q_list[i]))
                for j in range(len(self.r[i])):
                    print('r_{0}_{1} = {2}'.format(self.q_list[i], j, self.r[i][j]))
                    
                for j in range(len(self.x_s[i])):
                    print('x_{0} = {1}'.format(j, self.x_s[i][j]))
                    
            print('\nEquations:')
            for i in range(len(self.equations)):
                print('x = {0}mod{1}'.format(self.equations[i][0], self.equations[i][1]))

            print('\nSolution:\nM = {0}'.format(self.M))
            
            s = ''
            for i in range(len(self.m)):
                if i == len(self.m) - 1:
                    s += '{0}*{1}*{2}'.format(self.equations[i][0], self.m[i], self.N[i] % self.M)
                else:
                    s += '{0}*{1}*{2} + '.format(self.equations[i][0], self.m[i], self.N[i] % self.M)
                print('M_{0} = {1}, N_{0} = M_{0}^(-1)mod{2} = {1}^(-1)mod{2} = {3}'.format(i, self.m[i], self.equations[i][1], self.N[i]))
            
            print('X = ({0})mod{1} = {2}'.format(s, self.M, self.x))

In [387]:
# %%timeit
sph = SilverPohligHellman(3, 148, 181)
sph.do_all_stuff(verbose = True)

3^x = 148mod181
p - 1 = 2^2 + 3^2

##########
q = 2
r_2_0 = 1
r_2_1 = 1
x_0 = 1
x_1 = 1

##########
q = 3
r_3_0 = 1
r_3_1 = 132
r_3_2 = 48
x_0 = 2
x_1 = 1

Equations:
x = 3mod4
x = 5mod9

Solution:
M = 36
M_0 = 9, N_0 = M_0^(-1)mod4 = 9^(-1)mod4 = 1
M_1 = 4, N_1 = M_1^(-1)mod9 = 4^(-1)mod9 = -2
X = (3*9*1 + 5*4*34)mod36 = 23
